In [1]:
from xai.dqn_autoencoder import DQN
from xai.stream import Stream
from xai.bytes import GigaBytes
     

dqn = DQN(autoencoder_path="autoencoder-l32.pt",translate=True, rotate=True, device="cpu") 

In [2]:
dqn.train(
    total_time_steps=1_000_000,
    replay_buffer_size=GigaBytes(8),
    learning_rate = 1e-4,
    learning_starts = 1000,
    batch_size = 32,
    tau = 1.0,
    gamma = 0.99,
    train_frequency = (32, "step"),
    frame_skip=4,
    gradient_steps = 2,
    save_frequency= (3, "episode", "./dqn-model.pt"),
    target_update_frequency = (1, "episode"),
    final_exploration_rate_progress = 0.03,
    initial_exploration_rate = 1.0,
    final_exploration_rate = 0.05,
    verbose = True
)

  0%|          | 0/1000000 [00:00<?, ?it/s]CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
time_step=3511, episode=3, total_reward=0, exploration=0.89:   0%|          | 3512/1000000 [25:04<118:36:26,  2.33it/s]   


KeyboardInterrupt: 

In [16]:
from xai.window import Window
import numpy as np

with Window("Asteroids", 60, 4.0) as window:
    for step in dqn.rollout(0.0, frame_skips=1).take(3000):
        obs = step.observation.translated().rotated()
        tensor = obs.tensor(normalize=True, device="cpu").unsqueeze(0)
        numpy = obs.numpy(True)
        decode = dqn._autoencoder(tensor).squeeze(0).numpy(force=True)
        window(np.hstack([numpy, decode]))